In [221]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, f1_score, make_scorer
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb

##Carga de los datos

In [80]:
file_path = "/content/prueba_op_base_completa_entrenamiento_final_8a54228f_20241124T181021.csv"
df = pd.read_csv(file_path)

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 488022 entries, 0 to 488021
Data columns (total 92 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   nit_enmascarado                  488022 non-null  int64  
 1   num_oblig_orig_enmascarado       488022 non-null  int64  
 2   num_oblig_enmascarado            488022 non-null  int64  
 3   fecha_var_rpta_alt               488022 non-null  int64  
 4   var_rpta_alt                     488022 non-null  int64  
 5   tipo_var_rpta_alt                488022 non-null  object 
 6   banca                            488022 non-null  object 
 7   segmento                         488022 non-null  object 
 8   producto                         488022 non-null  object 
 9   producto_cons                    488022 non-null  object 
 10  aplicativo                       488022 non-null  object 
 11  min_mora                         488022 non-null  int64  
 12  ma

In [82]:
df.head(5)

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,fecha_var_rpta_alt,var_rpta_alt,tipo_var_rpta_alt,banca,segmento,producto,producto_cons,...,origen_fondos,f_vinc,cli_actualizado,segm,nicho,region_of,egresos_mes,tot_patrimonio,ciiu,smmlv
0,26,462460,617879,202310,0,e_cero_tipo_2,Banca Personas,Personal plus,TARJETA DE CREDITO,Tarjeta de Credito,...,OTROS,19890208.0,N,PLUS,-1,BOGOTA Y CUNDINAMARCA,4200000.0,157000000.0,ASALARIADOS,1160000
1,87,796610,283732,202311,1,a_uno_tipo_1,Banca Personas,Personal,LIBRE INVERSION,Libre Inversion,...,SALARIO Y DEMAS PAGOS LABORALES,20061027.0,N,PERSONAL,-1,CENTRO,100000.0,9042452.0,ASALARIADOS,1160000
2,89,964314,116028,202309,0,e_cero_tipo_2,Independientes,Micropyme,LIBRE INVERSION,Libre Inversion,...,OTROS,20030312.0,S,INDEPENDIENTES,-1,ANTIOQUIA,2200000.0,154398.0,"COMERCIO DE PARTES, PIEZAS (AUTOPARTES) Y ACCE...",1160000
3,171,825004,255338,202309,1,b_uno_tipo_2,Banca Personas,Personal,LIBRE INVERSION,Libre Inversion,...,SALARIO Y DEMAS PAGOS LABORALES,20070926.0,N,PERSONAL,-1,BOGOTA Y CUNDINAMARCA,600000.0,150000000.0,ASALARIADOS,1160000
4,187,94466,993313,202310,0,e_cero_tipo_2,Banca Personas,Personal,TARJETA DE CREDITO,Tarjeta de Credito,...,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0


##Revision cuantos duplicados y valores faltantes hay

In [83]:
print(f'\nMissing values: {df.isna().sum().sum()}')
print(f'Duplicated values: {df.duplicated().sum()}')


Missing values: 1404345
Duplicated values: 0


Funcion para imputar variables nulas y con NaN utilizando el percentil 75.

In [84]:
def impute_with_75th_percentile(df):
    for column in df.columns:
        if df[column].isnull().any():
            percentile_75 = np.percentile(df[column].dropna(), 75)
            df[column].fillna(percentile_75, inplace=True)
    return df

In [85]:
numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
non_numerical_columns = df.select_dtypes(include=['object']).columns.tolist()


print("\nNumerical Columns:", numerical_columns)
print("Categorical Columns:", non_numerical_columns)


Numerical Columns: ['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado', 'fecha_var_rpta_alt', 'var_rpta_alt', 'min_mora', 'max_mora', 'dias_mora_fin', 'vlr_obligacion', 'vlr_vencido', 'saldo_capital', 'endeudamiento', 'desc_alternativa1', 'desc_alternativa2', 'desc_alternativa3', 'cant_alter_posibles', 'cant_gestiones', 'cant_gestiones_binario', 'rpc', 'promesas_cumplidas', 'cant_promesas_cumplidas_binario', 'cant_acuerdo', 'cant_acuerdo_binario', 'valor_cuota_mes', 'pago_cuota', 'porc_pago_cuota', 'pago_mes', 'porc_pago_mes', 'lote', 'prob_propension', 'prob_alrt_temprana', 'prob_auto_cura', 'fecha_corte_t3', 'valor_cuota_mes_t3', 'pago_total', 'fecha_pago_minima', 'porc_pago', 'cod_tipo_doc', 'ano_nac_cli', 'edad_cli', 'num_hijos', 'personas_dependientes', 'total_ing', 'tot_activos', 'tot_pasivos', 'f_vinc', 'egresos_mes', 'tot_patrimonio', 'smmlv']
Categorical Columns: ['tipo_var_rpta_alt', 'banca', 'segmento', 'producto', 'producto_cons', 'aplicativo', 'rango

Esto para guardar el dataframe original antes de eliminar columnas.

In [ ]:
df = df_2

In [86]:
df_2 = df
df = df.drop(columns=non_numerical_columns)

In [87]:
df.head(5)

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,fecha_var_rpta_alt,var_rpta_alt,min_mora,max_mora,dias_mora_fin,vlr_obligacion,vlr_vencido,...,edad_cli,num_hijos,personas_dependientes,total_ing,tot_activos,tot_pasivos,f_vinc,egresos_mes,tot_patrimonio,smmlv
0,26,462460,617879,202310,0,5,16,16,664013.0,80113,...,91.0,0.0,0.0,8143300.0,182000000.0,25000000.0,19890208.0,4200000.0,157000000.0,1160000
1,87,796610,283732,202311,1,10,25,25,4554265.0,255388,...,53.0,0.0,0.0,1229325.0,9042452.0,0.0,20061027.0,100000.0,9042452.0,1160000
2,89,964314,116028,202309,0,39,67,67,1134596.0,100805,...,63.0,0.0,0.0,1286487.0,154398.0,0.0,20030312.0,2200000.0,154398.0,1160000
3,171,825004,255338,202309,1,2,177,21,7471765.0,823958,...,57.0,3.0,4.0,1800000.0,150000000.0,0.0,20070926.0,600000.0,150000000.0,1160000
4,187,94466,993313,202310,0,5,16,16,2824093.0,192461,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0


Realizo Imputacion

In [88]:
df_imputed = impute_with_75th_percentile(df)

<ipython-input-84-72ead7347e7a>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(percentile_75, inplace=True)


In [89]:
df_imputed.head(5)

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,fecha_var_rpta_alt,var_rpta_alt,min_mora,max_mora,dias_mora_fin,vlr_obligacion,vlr_vencido,...,edad_cli,num_hijos,personas_dependientes,total_ing,tot_activos,tot_pasivos,f_vinc,egresos_mes,tot_patrimonio,smmlv
0,26,462460,617879,202310,0,5,16,16,664013.0,80113,...,91.0,0.0,0.0,8143300.0,182000000.0,25000000.0,19890208.0,4200000.0,157000000.0,1160000
1,87,796610,283732,202311,1,10,25,25,4554265.0,255388,...,53.0,0.0,0.0,1229325.0,9042452.0,0.0,20061027.0,100000.0,9042452.0,1160000
2,89,964314,116028,202309,0,39,67,67,1134596.0,100805,...,63.0,0.0,0.0,1286487.0,154398.0,0.0,20030312.0,2200000.0,154398.0,1160000
3,171,825004,255338,202309,1,2,177,21,7471765.0,823958,...,57.0,3.0,4.0,1800000.0,150000000.0,0.0,20070926.0,600000.0,150000000.0,1160000
4,187,94466,993313,202310,0,5,16,16,2824093.0,192461,...,0.0,0.0,0.0,0.0,0.0,0.0,20190313.0,0.0,0.0,0


Revisamos nuevamente como quedaron las variables

In [90]:
print(f'\nMissing values: {df_imputed.isna().sum().sum()}')
print(f'Duplicated values: {df_imputed.duplicated().sum()}')


Missing values: 0
Duplicated values: 0


#Se selecciona la variable respuesta como Y, y el resto del dataframe como X

In [91]:
y = df['var_rpta_alt']
X = df.drop(columns=['var_rpta_alt', 'porc_pago'])

#Para revisar si existen valores muy grandes o infinitos.

In [92]:
print("¿Contiene valores infinitos?", np.isinf(X).any())
print("¿Contiene valores muy grandes?", (np.abs(X) > 1e308).any())

¿Contiene valores infinitos? nit_enmascarado                    False
num_oblig_orig_enmascarado         False
num_oblig_enmascarado              False
fecha_var_rpta_alt                 False
min_mora                           False
max_mora                           False
dias_mora_fin                      False
vlr_obligacion                     False
vlr_vencido                        False
saldo_capital                      False
endeudamiento                      False
desc_alternativa1                  False
desc_alternativa2                  False
desc_alternativa3                  False
cant_alter_posibles                False
cant_gestiones                     False
cant_gestiones_binario             False
rpc                                False
promesas_cumplidas                 False
cant_promesas_cumplidas_binario    False
cant_acuerdo                       False
cant_acuerdo_binario               False
valor_cuota_mes                    False
pago_cuota                  

Particionamiento de los datos, en 70% Entrenamiento y 30% Prueba.

In [113]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [114]:
print("Tamaño X entrenamiento", X_train.shape)
print("Tamaño y entrenamiento", y_train.shape)
print("Tamaño X prueba", X_test.shape)
print("Tamaño y prueba", y_test.shape)

Tamaño X entrenamiento (341615, 47)
Tamaño y entrenamiento (341615,)
Tamaño X prueba (146407, 47)
Tamaño y prueba (146407,)


##Se normalizan los datos utilizando StandarScaler

In [115]:
# Step 3: Standardize the features (Lasso is sensitive to scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Aplicamos Regresion Lasso para seleccion de variables

In [116]:
model_rg_lasso = LogisticRegression(penalty='l1', solver='liblinear', random_state=0)

# Definir los parámetros para la búsqueda
parameters = {'C': np.logspace(-3, 3, 100)}  # Rango de valores de C para probar

In [117]:
# Step 4: Apply Lasso regression for variable selection
alpha = 0.1  # Regularization strength; higher values increase sparsity
lasso = Lasso(alpha=alpha, random_state=42)
lasso.fit(X_train_scaled, y_train)

Lasso(alpha=0.1, random_state=42)

In [118]:
selected_features = np.where(lasso.coef_ != 0)[0]
eliminated_features = np.where(lasso.coef_ == 0)[0]

print("Variables seleccionadas (índices):", selected_features)
print("Variables seleccionadas (nombres):", X.columns[selected_features])
print("Variables eliminadas (índices):", eliminated_features)
print("Variables eliminadas (nombres):", X.columns[eliminated_features])

Variables seleccionadas (índices): [ 6 17 19 21 26 31]
Variables seleccionadas (nombres): Index(['dias_mora_fin', 'rpc', 'cant_promesas_cumplidas_binario',
       'cant_acuerdo_binario', 'porc_pago_mes', 'fecha_corte_t3'],
      dtype='object')
Variables eliminadas (índices): [ 0  1  2  3  4  5  7  8  9 10 11 12 13 14 15 16 18 20 22 23 24 25 27 28
 29 30 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
Variables eliminadas (nombres): Index(['nit_enmascarado', 'num_oblig_orig_enmascarado',
       'num_oblig_enmascarado', 'fecha_var_rpta_alt', 'min_mora', 'max_mora',
       'vlr_obligacion', 'vlr_vencido', 'saldo_capital', 'endeudamiento',
       'desc_alternativa1', 'desc_alternativa2', 'desc_alternativa3',
       'cant_alter_posibles', 'cant_gestiones', 'cant_gestiones_binario',
       'promesas_cumplidas', 'cant_acuerdo', 'valor_cuota_mes', 'pago_cuota',
       'porc_pago_cuota', 'pago_mes', 'lote', 'prob_propension',
       'prob_alrt_temprana', 'prob_auto_cura', 'valor_cuota_mes_t3',
 

Seleccionamos las mejores variables

In [119]:
X_train_selected = pd.DataFrame(X_train_scaled[:, selected_features], columns=X.columns[selected_features])
X_test_selected = pd.DataFrame(X_test_scaled[:, selected_features], columns=X.columns[selected_features])

In [120]:
X_train_selected.count()

,0
dias_mora_fin,341615
rpc,341615
cant_promesas_cumplidas_binario,341615
cant_acuerdo_binario,341615
porc_pago_mes,341615
fecha_corte_t3,341615


Obtenemos el error cuadrado.

In [121]:
y_pred = lasso.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Data: {mse:.4f}")

Mean Squared Error on Test Data: 0.2000


##Modelo de regresion logistica

In [122]:
log_reg = LogisticRegression(max_iter=1000)

In [123]:
param_grid_rl = {
    'C': [0.01, 0.1, 1, 10, 100],        # Inverso de la regularización L2
    'penalty': ['l2', 'none'],           # Tipo de regularización
    'solver': ['lbfgs', 'saga'],         # Solvers disponibles
    'fit_intercept': [True, False]       # Ajustar el intercepto
}

In [124]:
# Configuración del GridSearchCV
grid_search_rl = GridSearchCV(
    estimator=log_reg,
    param_grid=param_grid_rl,
    scoring='f1',  # Optimizar el F1-score
    cv=5,          # Número de particiones para validación cruzada
    n_jobs=-1,     # Paralelización
    verbose=1      # Mensajes de progreso
)

In [125]:
grid_search_rl.fit(X_train_selected, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
100 fits failed out of a total of 200.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100],
                         'fit_intercept': [True, False],
                         'penalty': ['l2', 'none'],
                         'solver': ['lbfgs', 'saga']},
             scoring='f1', verbose=1)

In [126]:
best_model_rl = grid_search_rl.best_estimator_
best_params_rl = grid_search_rl.best_params_
print("Mejores hiperparámetros encontrados:")
print(best_params_rl)

Mejores hiperparámetros encontrados:
{'C': 0.01, 'fit_intercept': False, 'penalty': 'l2', 'solver': 'lbfgs'}


In [130]:
# Evaluación en el conjunto de prueba
y_pred_rl = best_model_rl.predict(X_test_selected)
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_rl, digits=4))



Reporte de clasificación:
              precision    recall  f1-score   support

           0     0.7553    0.8950    0.8192     70618
           1     0.8818    0.7298    0.7986     75789

    accuracy                         0.8095    146407
   macro avg     0.8185    0.8124    0.8089    146407
weighted avg     0.8208    0.8095    0.8086    146407



In [132]:
f1_rl = f1_score(y_test, y_pred_rl)
print(f"F1-score en el conjunto de prueba: {f1_rl:.4f}")

F1-score en el conjunto de prueba: 0.7986


##Modelo de LigthGBM Classifier

In [133]:
lgb_model = lgb.LGBMClassifier(random_state=42)

In [134]:
param_grid_lgb = {
    'num_leaves': [15, 31, 50],        # Número de hojas en un árbol
    'max_depth': [-1, 5, 10],         # Profundidad máxima del árbol (-1 para ilimitado)
    'learning_rate': [0.01, 0.1, 0.2],  # Tasa de aprendizaje
    'n_estimators': [50, 100, 200],   # Número de árboles
    'min_child_samples': [10, 20, 30] # Tamaño mínimo de datos en una hoja
}

In [135]:
grid_search_lgb = GridSearchCV(
    estimator=lgb_model,
    param_grid=param_grid_lgb,
    scoring='f1',  # Optimizar el F1-score
    cv=5,          # Número de particiones para validación cruzada
    verbose=1,     # Mostrar el progreso
    n_jobs=-1      # Paralelización
)

In [136]:
grid_search_lgb.fit(X_train_selected, y_train)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[LightGBM] [Info] Number of positive: 177016, number of negative: 164599
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.034219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 436
[LightGBM] [Info] Number of data points in the train set: 341615, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.518174 -> initscore=0.072728
[LightGBM] [Info] Start training from score 0.072728


GridSearchCV(cv=5, estimator=LGBMClassifier(random_state=42), n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [-1, 5, 10],
                         'min_child_samples': [10, 20, 30],
                         'n_estimators': [50, 100, 200],
                         'num_leaves': [15, 31, 50]},
             scoring='f1', verbose=1)

In [148]:
best_model_lgb = grid_search_lgb.best_estimator_
best_params_lgb = grid_search_lgb.best_params_
print("Mejores hiperparámetros encontrados:")
print(best_params_lgb)

Mejores hiperparámetros encontrados:
{'learning_rate': 0.1, 'max_depth': 10, 'min_child_samples': 10, 'n_estimators': 200, 'num_leaves': 15}


In [149]:
y_pred_lgb = best_model_lgb.predict(X_test_selected)

In [150]:
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_lgb, digits=4))


Reporte de clasificación:
              precision    recall  f1-score   support

           0     0.8247    0.8885    0.8554     70618
           1     0.8880    0.8240    0.8548     75789

    accuracy                         0.8551    146407
   macro avg     0.8563    0.8562    0.8551    146407
weighted avg     0.8575    0.8551    0.8551    146407



In [151]:
f1 = f1_score(y_test, y_pred_lgb)
print(f"F1-score en el conjunto de prueba: {f1:.4f}")

F1-score en el conjunto de prueba: 0.8548


##Decision Tree Classifier

In [230]:
dt = DecisionTreeClassifier(random_state=42)

In [231]:
param_grid_tree = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [222]:
f1_scorer = make_scorer(f1_score, average='weighted')

In [232]:
grid_search_tree = GridSearchCV(estimator=dt, param_grid=param_grid_tree,
                           scoring=f1_scorer, cv=5, verbose=1, n_jobs=-1)

In [233]:
grid_search_tree.fit(X_train_selected, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 5, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring=make_scorer(f1_score, response_method='predict', average=weighted),
             verbose=1)

In [234]:
best_model_tree = grid_search_tree.best_estimator_
best_params_tree = grid_search_tree.best_params_
print("Mejores hiperparámetros encontrados:")
print(best_params_tree)

Mejores hiperparámetros encontrados:
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}


In [235]:
y_pred_tree = best_model_tree.predict(X_test_selected)

In [236]:
print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred_tree, digits=4))


Reporte de clasificación:
              precision    recall  f1-score   support

           0     0.8232    0.8900    0.8553     70618
           1     0.8891    0.8219    0.8542     75789

    accuracy                         0.8548    146407
   macro avg     0.8562    0.8560    0.8548    146407
weighted avg     0.8573    0.8548    0.8547    146407



In [237]:
f1_tree = f1_score(y_test, y_pred_tree)
print(f"F1-score en el conjunto de prueba: {f1_tree:.4f}")

F1-score en el conjunto de prueba: 0.8542


##Carga de la base a inferir

In [154]:
file_inferior = "/content/Clientes a Inferir.csv"
df_inferir = pd.read_csv(file_path)

In [155]:
df_inferir.head(5)

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,fecha_var_rpta_alt,var_rpta_alt,tipo_var_rpta_alt,banca,segmento,producto,producto_cons,...,origen_fondos,f_vinc,cli_actualizado,segm,nicho,region_of,egresos_mes,tot_patrimonio,ciiu,smmlv
0,26,462460,617879,202310,0,e_cero_tipo_2,Banca Personas,Personal plus,TARJETA DE CREDITO,Tarjeta de Credito,...,OTROS,19890208.0,N,PLUS,-1,BOGOTA Y CUNDINAMARCA,4200000.0,157000000.0,ASALARIADOS,1160000
1,87,796610,283732,202311,1,a_uno_tipo_1,Banca Personas,Personal,LIBRE INVERSION,Libre Inversion,...,SALARIO Y DEMAS PAGOS LABORALES,20061027.0,N,PERSONAL,-1,CENTRO,100000.0,9042452.0,ASALARIADOS,1160000
2,89,964314,116028,202309,0,e_cero_tipo_2,Independientes,Micropyme,LIBRE INVERSION,Libre Inversion,...,OTROS,20030312.0,S,INDEPENDIENTES,-1,ANTIOQUIA,2200000.0,154398.0,"COMERCIO DE PARTES, PIEZAS (AUTOPARTES) Y ACCE...",1160000
3,171,825004,255338,202309,1,b_uno_tipo_2,Banca Personas,Personal,LIBRE INVERSION,Libre Inversion,...,SALARIO Y DEMAS PAGOS LABORALES,20070926.0,N,PERSONAL,-1,BOGOTA Y CUNDINAMARCA,600000.0,150000000.0,ASALARIADOS,1160000
4,187,94466,993313,202310,0,e_cero_tipo_2,Banca Personas,Personal,TARJETA DE CREDITO,Tarjeta de Credito,...,0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0


In [156]:
numerical_columns_inf = df_inferir.select_dtypes(include=['int64', 'float64']).columns.tolist()
non_numerical_columns_inf = df_inferir.select_dtypes(include=['object']).columns.tolist()

print("\nNumerical Columns:", numerical_columns_inf)
print("Categorical Columns:", non_numerical_columns_inf)


Numerical Columns: ['nit_enmascarado', 'num_oblig_orig_enmascarado', 'num_oblig_enmascarado', 'fecha_var_rpta_alt', 'var_rpta_alt', 'min_mora', 'max_mora', 'dias_mora_fin', 'vlr_obligacion', 'vlr_vencido', 'saldo_capital', 'endeudamiento', 'desc_alternativa1', 'desc_alternativa2', 'desc_alternativa3', 'cant_alter_posibles', 'cant_gestiones', 'cant_gestiones_binario', 'rpc', 'promesas_cumplidas', 'cant_promesas_cumplidas_binario', 'cant_acuerdo', 'cant_acuerdo_binario', 'valor_cuota_mes', 'pago_cuota', 'porc_pago_cuota', 'pago_mes', 'porc_pago_mes', 'lote', 'prob_propension', 'prob_alrt_temprana', 'prob_auto_cura', 'fecha_corte_t3', 'valor_cuota_mes_t3', 'pago_total', 'fecha_pago_minima', 'porc_pago', 'cod_tipo_doc', 'ano_nac_cli', 'edad_cli', 'num_hijos', 'personas_dependientes', 'total_ing', 'tot_activos', 'tot_pasivos', 'f_vinc', 'egresos_mes', 'tot_patrimonio', 'smmlv']
Categorical Columns: ['tipo_var_rpta_alt', 'banca', 'segmento', 'producto', 'producto_cons', 'aplicativo', 'rango

In [157]:
df_2_inf = df_inferir
df_inferir = df_inferir.drop(columns=non_numerical_columns_inf)

In [146]:
df_inferir.head(5)

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,fecha_var_rpta_alt,var_rpta_alt,min_mora,max_mora,dias_mora_fin,vlr_obligacion,vlr_vencido,...,edad_cli,num_hijos,personas_dependientes,total_ing,tot_activos,tot_pasivos,f_vinc,egresos_mes,tot_patrimonio,smmlv
0,26,462460,617879,202310,0,5,16,16,664013.0,80113,...,91.0,0.0,0.0,8143300.0,182000000.0,25000000.0,19890208.0,4200000.0,157000000.0,1160000
1,87,796610,283732,202311,1,10,25,25,4554265.0,255388,...,53.0,0.0,0.0,1229325.0,9042452.0,0.0,20061027.0,100000.0,9042452.0,1160000
2,89,964314,116028,202309,0,39,67,67,1134596.0,100805,...,63.0,0.0,0.0,1286487.0,154398.0,0.0,20030312.0,2200000.0,154398.0,1160000
3,171,825004,255338,202309,1,2,177,21,7471765.0,823958,...,57.0,3.0,4.0,1800000.0,150000000.0,0.0,20070926.0,600000.0,150000000.0,1160000
4,187,94466,993313,202310,0,5,16,16,2824093.0,192461,...,0.0,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0


In [159]:
X_inferencia = df_inferir[['dias_mora_fin', 'rpc', 'cant_promesas_cumplidas_binario', 'cant_acuerdo_binario', 'porc_pago_mes', 'fecha_corte_t3']]

##Realizamos prediccion con Mejor Modelo

In [160]:
y_pred_fin = best_model_lgb.predict(X_inferencia)

#Obtenemos probabilidad

In [161]:
y_pred_proba = best_model_lgb.predict_proba(X_inferencia)  # Probabilidades por clase

In [163]:
print("Predicciones:")
print(y_pred_fin)

Predicciones:
[1 1 1 ... 1 1 1]


In [177]:
print("\nProbabilidades de las predicciones:")
print(pd.DataFrame(y_pred_proba, columns=[f'Clase_{i}' for i in range(y_pred_proba.shape[1])]))


Probabilidades de las predicciones:
         Clase_0   Clase_1
0       0.293370  0.706630
1       0.293370  0.706630
2       0.412449  0.587551
3       0.412449  0.587551
4       0.293370  0.706630
...          ...       ...
488017  0.412449  0.587551
488018  0.293370  0.706630
488019  0.293370  0.706630
488020  0.293370  0.706630
488021  0.412449  0.587551

[488022 rows x 2 columns]


In [199]:
df_predecido = pd.DataFrame(y_pred_fin, columns=['var_rpta_alt'])

In [200]:
y_predecido = pd.DataFrame(y_pred_proba, columns=['Clase_0', 'Prob_uno'])

##Generacion base final

In [210]:
df_inferir['ID'] = (
    df_inferir['nit_enmascarado'].astype(str) + '#' +
    df_inferir['num_oblig_orig_enmascarado'].astype(str) + '#' +
    df_inferir['num_oblig_orig_enmascarado'].astype(str)
)

# Si necesitas reasignar a un nuevo DataFrame:
df_inferido = df_inferir

In [211]:
df_inferido.head(5)

,nit_enmascarado,num_oblig_orig_enmascarado,num_oblig_enmascarado,fecha_var_rpta_alt,var_rpta_alt,min_mora,max_mora,dias_mora_fin,vlr_obligacion,vlr_vencido,...,personas_dependientes,total_ing,tot_activos,tot_pasivos,f_vinc,egresos_mes,tot_patrimonio,smmlv,concatenado,ID
0,26,462460,617879,202310,0,5,16,16,664013.0,80113,...,0.0,8143300.0,182000000.0,25000000.0,19890208.0,4200000.0,157000000.0,1160000,26#462460#462460,26#462460#462460
1,87,796610,283732,202311,1,10,25,25,4554265.0,255388,...,0.0,1229325.0,9042452.0,0.0,20061027.0,100000.0,9042452.0,1160000,87#796610#796610,87#796610#796610
2,89,964314,116028,202309,0,39,67,67,1134596.0,100805,...,0.0,1286487.0,154398.0,0.0,20030312.0,2200000.0,154398.0,1160000,89#964314#964314,89#964314#964314
3,171,825004,255338,202309,1,2,177,21,7471765.0,823958,...,4.0,1800000.0,150000000.0,0.0,20070926.0,600000.0,150000000.0,1160000,171#825004#825004,171#825004#825004
4,187,94466,993313,202310,0,5,16,16,2824093.0,192461,...,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0,187#94466#94466,187#94466#94466


#Concatenamos segun estructura deseada

In [212]:
df_final = pd.concat([df_inferido['ID'], df_predecido, y_predecido], axis=1)

In [213]:
df_final = df_final[['ID', 'var_rpta_alt', 'Prob_uno']]

In [214]:
df_final.head(5)

,ID,var_rpta_alt,Prob_uno
0,26#462460#462460,1,0.706630
1,87#796610#796610,1,0.706630
2,89#964314#964314,1,0.587551
3,171#825004#825004,1,0.587551
4,187#94466#94466,1,0.706630


##Exportamos los resultados

In [215]:
df_final.to_csv('/content/Resultados_Modelo.csv', index=False)